# The OAuth 2.0 Authorization Framework

This notebook contains information taken directly from [**The OAuth 2.0 Authorization Framework** (RFC 6749)](https://tools.ietf.org/html/rfc6749). Once I complete this document I'll procceed to describe and explain OAuth 2.0 using my own words.

---

### Introduction

##### Roles

OAuth defines four roles:

* **Resource owner:** An entity capable of granting access to a protected resource. When the resource owner is a person, it is referred to as an *end-user*.
* **Resource server:** The server hosting the protected resource, capable of accepting and responding to protected resource requests using *access tokens*.
* **Client:** An application making protected resource requests on behalf of the resource owner and with its authorization. The term "client" does not imply any particular implementation characteristics (e.g., whether the application executes on a server, a desktop, or other devices).
* **Authorization server:** The server issuing access tokens to the client after successfully authenticating the resource owner and obtaining authorization.

The interaction between the authorization server and resource server is beyond the scope of this specification. The authorization server may be the same server as the resource server or a separate entity. A single authorization server may issue access tokens accepted by multiple resource servers.

##### Protocol Flow

     +--------+                               +---------------+
     |        |--(A)- Authorization Request ->|   Resource    |
     |        |                               |     Owner     |
     |        |<-(B)-- Authorization Grant ---|               |
     |        |                               +---------------+
     |        |
     |        |                               +---------------+
     |        |--(C)-- Authorization Grant -->| Authorization |
     | Client |                               |     Server    |
     |        |<-(D)----- Access Token -------|               |
     |        |                               +---------------+
     |        |
     |        |                               +---------------+
     |        |--(E)----- Access Token ------>|    Resource   |
     |        |                               |     Server    |
     |        |<-(F)--- Protected Resource ---|               |
     +--------+                               +---------------+
     

* (A) The client requests authorization from the resource owner.  The authorization request can be made directly to the resource owner (as shown), or preferably indirectly via the authorization server as an intermediary.
* (B) The client receives an authorization grant, which is a credential representing the resource owner's authorization, expressed using one of four grant types defined in this specification or using an extension grant type. The authorization grant type depends on the method used by the client to request authorization and the types supported by the authorization server.
* (C) The client requests an access token by authenticating with the authorization server and presenting the authorization grant.
* (D) The authorization server authenticates the client and validates the authorization grant, and if valid, issues an access token.
* (E) The client requests the protected resource from the resource server and authenticates by presenting the access token.
* (F) The resource server validates the access token, and if valid, serves the request.

The preferred method for the client to obtain an authorization grant from the resource owner (depicted in steps (A) and (B)) is to **use the authorization server as an intermediary** (...).

##### Access Token\*

Access tokens are credentials used to access protected resources.  An access token is a string representing an authorization issued to the client.  The string is usually opaque to the client.  Tokens represent specific scopes and durations of access, granted by the resource owner, and enforced by the resource server and authorization server.

The token may denote an identifier used to retrieve the authorization information or may self-contain the authorization information in a verifiable manner (i.e., a token string consisting of some data and a signature).  Additional authentication credentials, which are beyond the scope of this specification, may be required in order for the client to use a token.

The access token provides an abstraction layer, replacing different authorization constructs (e.g., username and password) with a single token understood by the resource server.  This abstraction enables issuing access tokens more restrictive than the authorization grant used to obtain them, as well as removing the resource server's need to understand a wide range of authentication methods.

Access tokens can have different formats, structures, and methods of utilization (e.g., cryptographic properties) based on the resource server security requirements.  Access token attributes and the methods used to access protected resources are beyond the scope of this specification and are defined by companion specifications such as [**The OAuth 2.0 Authorization Framework: Bearer Token Usage** (RFC6750)](https://tools.ietf.org/html/rfc6750).

> __\*__ There's a different section about Refresh Tokens

---

### Client Registration

Before initiating the protocol, the client registers with the authorization server.  Client registration does not require a direct interaction between the client and the authorization server.  When supported by the authorization server, registration can rely on other means for establishing trust and obtaining the required client properties (e.g., redirection URI, client type).  For example, registration can be accomplished using a self-issued or third-party-issued assertion, or by the authorization server performing client discovery using a trusted channel.

When registering a client, the client developer SHALL:
 
 * specify the client type as described in Section 2.1,
 * provide its client redirection URIs as described in Section 3.1.2, and
 * include any other information required by the authorization server (e.g., application name, website, description, logo image, the acceptance of legal terms).
 
 ##### Client Types
 
OAuth defines two client types, based on their ability to authenticate securely with the authorization server (i.e., ability to maintain the confidentiality of their client credentials):
 
* **Confidential:** Clients capable of maintaining the confidentiality of their credentials (e.g., client implemented on a secure server with restricted access to the client credentials), or capable of secure client authentication using other means.

* **Public:** Clients incapable of maintaining the confidentiality of their credentials (e.g., clients executing on the device used by the resource owner, such as an installed native application or a web browser-based application), and incapable of secure client authentication via any other means.

This specification has been designed around the following client profiles:

* **Web application:** A web application is a confidential client running on a web server.  Resource owners access the client via an HTML user interface rendered in a user-agent on the device used by the resource owner.  The client credentials as well as any access token issued to the client are stored on the web server and are not exposed to or accessible by the resource owner.

* **User-agent based application:** A user-agent-based application is a public client in which the client code is downloaded from a web server and executes within a user-agent (e.g., web browser) on the device used by the resource owner.  Protocol data and credentials are easily accessible (and often visible) to the resource owner.  Since such applications reside within the user-agent, they can make seamless use of the user-agent capabilities when requesting authorization.

* **Native application:** A native application is a public client installed and executed on the device used by the resource owner.  Protocol data and credentials are accessible to the resource owner.  It is assumed that any client authentication credentials included in the application can be extracted.  On the other hand, dynamically issued credentials such as access tokens or refresh tokens can receive an acceptable level of protection.  At a minimum, these credentials are protected from hostile servers with which the application may interact.  On some platforms, these credentials might be protected from other applications residing on the same device.

##### Client Identifier

The authorization server issues the registered client a client identifier -- a unique string representing the registration information provided by the client.  The client identifier is not a secret; it is exposed to the resource owner and MUST NOT be used alone for client authentication.  The client identifier is unique to the authorization server.

##### Client Authentication

If the client type is confidential, the client and authorization server establish a client authentication method suitable for the security requirements of the authorization server.  The authorization server MAY accept any form of client authentication meeting its security requirements.

Confidential clients are typically issued (or establish) a set of client credentials used for authenticating with the authorization server (e.g., password, public/private key pair).

The authorization server MUST support the HTTP Basic authentication scheme for authenticating clients that were issued a client password. Alternatively, the authorization server MAY support including the client credentials in the request-body using the **client_id** and **client_secret** parameters. 

Including the client credentials in the request-body using the two parameters is NOT RECOMMENDED and SHOULD be limited to clients unable to directly utilize the HTTP Basic authentication scheme (or other password-based HTTP authentication schemes).  The parameters can only be transmitted in the request-body and MUST NOT be included in the request URI.

Since this client authentication method involves a password, the authorization server MUST protect any endpoint utilizing it against brute force attacks.



### Protocol Endpoints